In [34]:
import requests

In [78]:
parameters = {
    "name":"Austria"
}
response1 = requests.get("https://restcountries.eu/rest/v2/region/europe?fields=name;nativeName;alpha2Code;alpha3Code;latlng;population;area;capital;languages;topLevelDomain;timezones;regionalBlocs;currencies",params=parameters)

print(response1.status_code)

eu_json = response1.json()

200


In [153]:
listka = ["name","nativeName","alpha2Code","alpha3Code","latlng","population","area","capital","languages","topLevelDomain","timezones","regionalBlocs","currencies"]
name=[]
nativeName=[]
alpha2Code=[]
alpha3Code=[]
latlng=[]
population=[]
area=[]
capital=[]
languages=[]
topLevelDomain=[]
timezones=[]
regionalBlocs=[]
currencies=[]
for x in eu_json:
    name.append(x["name"])
    nativeName.append(x["nativeName"])
    alpha2Code.append(x["alpha2Code"])
    alpha3Code.append(x["alpha3Code"])
    latlng.append(x["latlng"])
    population.append(x["population"])
    try:
        area.append(x["area"])
    except KeyError:
        area.append("")
    capital.append(x["capital"])
    languages.append(x["languages"][0]["name"])
    topLevelDomain.append(x["topLevelDomain"][0])
    timezones.append(x["timezones"])
    if x["regionalBlocs"]==[]:
        regionalBlocs.append("No")
    elif "EU" not in x["regionalBlocs"][0].values() and "European Union" not in x["regionalBlocs"][0].values():
        regionalBlocs.append("No")
    else:
        regionalBlocs.append("Yes")
    currencies.append(x["currencies"]) 
print(len(area),len(population),len(capital),len(regionalBlocs))

53 53 53 53


In [154]:
import pandas as pd 

pd.DataFrame({"name":name,"nativeName":nativeName,"alpha2Code":alpha2Code,"alpha3Code":alpha3Code,"latlng":latlng,"population":population,"area":area,"capital":capital,"languages":languages,"topLevelDomain":topLevelDomain,"timezones":timezones,"Is the country in EU?":regionalBlocs,"currencies":currencies})

,name,nativeName,alpha2Code,alpha3Code,latlng,population,area,capital,languages,topLevelDomain,timezones,Is the country in EU?,currencies
0,Åland Islands,Åland,AX,ALA,"[60.116667, 19.9]",28875,1580,Mariehamn,Swedish,.ax,[UTC+02:00],Yes,"[{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}]"
1,Albania,Shqipëria,AL,ALB,"[41.0, 20.0]",2886026,28748,Tirana,Albanian,.al,[UTC+01:00],No,"[{'code': 'ALL', 'name': 'Albanian lek', 'symb..."
2,Andorra,Andorra,AD,AND,"[42.5, 1.5]",78014,468,Andorra la Vella,Catalan,.ad,[UTC+01:00],No,"[{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}]"
3,Austria,Österreich,AT,AUT,"[47.33333333, 13.33333333]",8725931,83871,Vienna,German,.at,[UTC+01:00],Yes,"[{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}]"
4,Belarus,Белару́сь,BY,BLR,"[53.0, 28.0]",9498700,207600,Minsk,Belarusian,.by,[UTC+03:00],No,"[{'code': 'BYN', 'name': 'New Belarusian ruble..."
5,Belgium,België,BE,BEL,"[50.83333333, 4.0]",11319511,30528,Brussels,Dutch,.be,[UTC+01:00],Yes,"[{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}]"
6,Bosnia and Herzegovina,Bosna i Hercegovina,BA,BIH,"[44.0, 18.0]",3531159,51209,Sarajevo,Bosnian,.ba,[UTC+01:00],No,"[{'code': 'BAM', 'name': 'Bosnia and Herzegovi..."
7,Bulgaria,България,BG,BGR,"[43.0, 25.0]",7153784,110879,Sofia,Bulgarian,.bg,[UTC+02:00],Yes,"[{'code': 'BGN', 'name': 'Bulgarian lev', 'sym..."
8,Croatia,Hrvatska,HR,HRV,"[45.16666666, 15.5]",4190669,56594,Zagreb,Croatian,.hr,[UTC+01:00],Yes,"[{'code': 'HRK', 'name': 'Croatian kuna', 'sym..."
9,Cyprus,Κύπρος,CY,CYP,"[35.0, 33.0]",847000,9251,Nicosia,Greek (modern),.cy,[UTC+02:00],Yes,"[{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}]"


In [131]:
eu_json[0]["languages"][0]["name"]

'Swedish'

In [134]:
cos = [{'acronym': 'EU', 'name': 'European Union'}]

In [140]:
"European Union" in cos[0].values()

True